<a href="https://colab.research.google.com/github/yenchunR/NLP_HW4/blob/master/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Module

In [10]:
!pip3 install hanziconv

     |████████████████████████████████| 276 kB 5.0 MB/s 
  Created wheel for hanziconv: filename=hanziconv-0.3.2-py2.py3-none-any.whl size=23225 sha256=d6e26247a0d6916053762a2b27e5079e6bd7e17ac85cb50a9938f0a9943d76ed
  Stored in directory: /root/.cache/pip/wheels/bf/e3/22/7bf50146a3ee95d1fdcbfabc44a1fe15b6e2ab7348ab7337bf
Successfully built hanziconv


In [91]:
!pip3 install gensim

# 將Colab連結Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 讀取所有檔案路徑

In [22]:
import pandas as pd
import os

# 把列出資料夾的程式碼寫成一個函式
filePath = []
def show_folder_content(folder_path):

    folder_content = os.listdir(folder_path)
    for item in folder_content:
        if os.path.isdir(folder_path + '/' + item):
            # 呼叫自己處理這個子資料夾
            show_folder_content(folder_path + '/' + item)
        elif os.path.isfile(folder_path + '/' + item):
            filePath.append(folder_path + '/' + item)
        else:
            print('無法辨識：' + item)


# 顯示指定資料夾的內容
target_folder = '/content/drive/My Drive/wiki_zh'
show_folder_content(target_folder)
print("Finish", len(filePath))
print(filePath)

Finish 1279
['/content/drive/My Drive/wiki_zh/AM/wiki_73', '/content/drive/My Drive/wiki_zh/AM/wiki_42', '/content/drive/My Drive/wiki_zh/AM/wiki_45', '/content/drive/My Drive/wiki_zh/AM/wiki_11', '/content/drive/My Drive/wiki_zh/AM/wiki_16', '/content/drive/My Drive/wiki_zh/AM/wiki_29', '/content/drive/My Drive/wiki_zh/AM/wiki_20', '/content/drive/My Drive/wiki_zh/AM/wiki_27', '/content/drive/My Drive/wiki_zh/AM/wiki_18', '/content/drive/My Drive/wiki_zh/AM/wiki_44', '/content/drive/My Drive/wiki_zh/AM/wiki_43', '/content/drive/My Drive/wiki_zh/AM/wiki_72', '/content/drive/My Drive/wiki_zh/AM/wiki_26', '/content/drive/My Drive/wiki_zh/AM/wiki_19', '/content/drive/My Drive/wiki_zh/AM/wiki_21', '/content/drive/My Drive/wiki_zh/AM/wiki_17', '/content/drive/My Drive/wiki_zh/AM/wiki_28', '/content/drive/My Drive/wiki_zh/AM/wiki_10', '/content/drive/My Drive/wiki_zh/AM/wiki_32', '/content/drive/My Drive/wiki_zh/AM/wiki_35', '/content/drive/My Drive/wiki_zh/AM/wiki_03', '/content/drive/My Dr

# jieba 斷詞

In [7]:
# coding=utf-8
import jieba 
import json
import re

dictionaryPath = '/content/drive/My Drive/wiki_zh/DriveUploader/dict.txt.big'
jieba.set_dictionary()##加入繁體中文

# 讀取停用詞
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords 

def LoadToSeg(filepath, fW):
    with open(filePath[0], 'r') as obj:
        for line in obj.readlines():
            dic = json.loads(line)
            temp_text = re.findall('[\u4e00-\u9fa5]', dic['text'])
            text = ''.join(temp_text)
            tags = jieba.lcut(text, cut_all=False)
            ThisLine = ''
            for tag in tags:
                if tag not in stopwords:  
                    if tag != '\t' and tag != '\n' and tag != '' and len(tag) > 1:  
                        ThisLine += tag  
                        ThisLine += " "   #再次組合成【帶空格】的串
            fW.write((ThisLine+"\n").encode('utf-8'))
            
stopwords = stopwordslist('/content/drive/My Drive/wiki_zh/DriveUploader/cn_stopwords.txt')  # 這裏加載停用詞的路徑 
fileSegWordDonePath ='/content/drive/My Drive/wiki_zh/DriveUploader/corpusSegDone.txt'
with open(fileSegWordDonePath,'wb') as fW:
    fW.truncate(0)
    index = 0
    for path in filePath:
        LoadToSeg(path, fW)
        index += 1
        if index % 100 == 0:
            print(index)
print("Finish")

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.037 seconds.
Prefix dict has been built successfully.


100
200
300
400
500
600
700
800
900
1000
1100
1200
Finish


# word2vec 轉成高維空間向量

In [8]:
from gensim.models import word2vec
# jieba分詞轉word2vec向量
fileSegWordDonePath ='/content/drive/My Drive/wiki_zh/DriveUploader/corpusSegDone.txt'
fileSegWordDonePath2 ='/content/drive/My Drive/wiki_zh/DriveUploader/corpusSegDone.model'

sentences = word2vec.LineSentence(fileSegWordDonePath)
model = word2vec.Word2Vec(sentences, size=250, min_count=1, negative=5, workers=3, sg=1)
model.save(fileSegWordDonePath2)
print("Finish")

Finish


# Result

In [29]:
# -*- coding: utf-8 -*-# -*- coding: utf-8 -*-
import sys
from gensim.models import word2vec 
from gensim import models
from hanziconv import HanziConv

def main():   
    a = input("input:") 
    Simp = HanziConv.toSimplified(a)
    model = models.Word2Vec.load(fileSegWordDonePath2)
    try:
        res = model.most_similar(Simp, topn=20)
        for item in res: 
            print(HanziConv.toTraditional(item[0]) + ':' + str(item[1]))
    except KeyError:
        print(a + " is not in the vocabulary")

if __name__=='__main__':
    main()

input:日本
火腿:0.45510420203208923
鬥士:0.4525272250175476
近畿:0.4509308338165283
堀隼平:0.4506727457046509
保守勢力:0.44793859124183655
東北大學:0.44748449325561523
四國:0.4469233751296997
島田海:0.44480088353157043
區北:0.44373926520347595
眾議院:0.4428616762161255
傲嬌:0.4428328573703766
旅遊觀光:0.44219428300857544
平深:0.44214490056037903
颱後:0.4413280189037323
武獅:0.44118741154670715
區南:0.440742552280426
福岡:0.43927642703056335
小崎亞:0.4392372965812683
初代社:0.43874943256378174
男友:0.43854284286499023


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if sys.path[0] == '':
